<a href="https://colab.research.google.com/github/yeoyujie/cmpsc448/blob/main/CMPSC448.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Data Preprocessing**

### Import all the necessary packages

In [ ]:
# Import the necessary libraries

import pandas as pd
import gzip

from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import cross_val_score
from sklearn.svm import SVC

import numpy as np

import nltk
import matplotlib.pyplot as plt
import seaborn as sns


In [ ]:
!ls


sample_data  train.txt.gz  unlabeled_test_test.txt


### Load the data file

In [ ]:
# Unzipping the file
with gzip.open('train.txt.gz', 'rt') as f:
    data = f.read()

# Splitting the data into lines
lines = data.split('\n')

# Splitting each line into its components and removing lines that are empty
lines = [line.split() for line in lines if line]

# Creating a DataFrame
df = pd.DataFrame(lines, columns=['Token', 'POS tag', 'Chunking tag'])

# Dropping the 'Chunking tag' column as it's not needed for this project
df = df.drop('Chunking tag', axis=1)


 #### Check for missing and duplicate values.





In [ ]:
# Check for missing values
print(df.isnull().sum())

# Drop duplicates
df = df.drop_duplicates()

# If there are missing values in 'Token' or 'POS tag' columns, we drop those rows
df = df.dropna(subset=['Token', 'POS tag'])

Token      0
POS tag    0
dtype: int64


In [ ]:
## For checking, remove later
print(df)
df.to_csv('df.txt', index=False, sep='\t')

             Token POS tag
0       Confidence      NN
1               in      IN
2              the      DT
3            pound      NN
4               is     VBZ
...            ...     ...
211688       2,480      CD
211689        cots     NNS
211696       pints     NNS
211698      Type-O      JJ
211715    Huricane     NNP

[20939 rows x 2 columns]


## 2. Split the data and vectorisation

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer()
X = tfidf_vectorizer.fit_transform(df['Token'])


In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['POS tag'])

In [ ]:
# Split the data into training and testing sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)


# **3. Model Training**


## Bayesian Classifier

In [ ]:
# Initialize the Multinomial Naive Bayes classifier
nb = MultinomialNB()

# Perform 5-fold cross validation
scores = cross_val_score(nb, X_train, y_train, cv=5)

print("Cross-validation scores: ", scores)
print("Mean cross-validation score: ", scores.mean())

# Fit the model to the training data
nb.fit(X_train, y_train)

# Predict the labels of the validation set
y_val_pred = nb.predict(X_val)

# Print the performance metrics
print(classification_report(y_val, y_val_pred))



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


Cross-validation scores:  [0.27364966 0.28119403 0.2841791  0.28925373 0.28358209]
Mean cross-validation score:  0.28237172240854813
              precision    recall  f1-score   support

           0       0.00      0.00      0.00         1
           5       0.00      0.00      0.00         1
           8       0.00      0.00      0.00         5
           9       0.96      0.66      0.79       378
          10       0.00      0.00      0.00         6
          12       0.00      0.00      0.00         4
          13       0.00      0.00      0.00        28
          14       0.46      0.28      0.35       572
          15       0.00      0.00      0.00        10
          16       0.00      0.00      0.00        11
          17       0.00      0.00      0.00         3
          18       0.25      0.12      0.16       761
          19       0.22      0.81      0.35       864
          20       0.00      0.00      0.00        26
          21       0.59      0.13      0.22       408
  

/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
#from sklearn.metrics import accuracy_score
# Initialize the model
#model = LogisticRegression(max_iter=1000)

# Train the model
#model.fit(X_train_tfidf, y_train)
#y_val_pred_lin = lin.predict(X_train_tfidf[:2, :])
#accuracy = accuracy_score(y_val, y_val_pred_lin)

# Train the Logistic model with maximum iteration of 1000
#lin = LogisticRegression(max_iter=2000, C=0.001).fit(X_train_tfidf, y_train)


#y_val_pred_lin = lin.predict(X_val_tfidf)



#print(classification_report(y_val, y_val_pred_lin))

#new
from sklearn.model_selection import RandomizedSearchCV

from scipy.stats import uniform, randint

# Define the model
model = LogisticRegression(max_iter = 1000)

# Define the parameters to tune
param_dist = {
    'C': uniform(0.001, 1000),  # Randomly sample C from a uniform distribution
    'max_iter': randint(1000, 2000)  # Randomly sample max_iter from a uniform distribution
}

# Initialize a RandomizedSearchCV object that will perform the hyperparameter tuning
random_search = RandomizedSearchCV(model, param_distributions=param_dist, n_iter=5, cv=3,)

# Use a smaller dataset for hyperparameter tuning
#X_train_tfidf_small = X_train[:1000]  # Example: Use the first 1000 samples

# Fit the data to perform hyperparameter tuning with a smaller dataset
random_search.fit(X_train, y_train)

# Fit the data to perform hyperparameter tuning
#random_search.fit(X_train_tfidf, y_train)

# Print the best parameters
print("Best Parameters: ", random_search.best_params_)

# Use the best model to make predictions
best_model = random_search.best_estimator_
y_val_pred_lin = best_model.predict(X_val)

# Print classification report
print(classification_report(y_val, y_val_pred_lin))



/usr/local/lib/python3.10/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=3.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modu

## Support Vector Machines

In [ ]:
# Model Initialization
svm_classifier = SVC(kernel='linear', probability=True, random_state=42)

# Training the SVM on the training data
svm_classifier.fit(X_train, y_train)

# Evaluate the trained SVM on the validation data
# Predict the labels of the validation set
y_val_pred_svm = svm_classifier.predict(X_val)

# Print the performance metrics
print(classification_report(y_val, y_val_pred_svm))


NameError: ignored

In [ ]:
## Better SVM with grid search optimisation

# Define the model
model = SVC(probability=True, random_state=42)

# Define the parameters to tune
parameters = {'C': [0.1, 1, 10],
              'kernel': ['linear', 'rbf'],
              'gamma': [0.1, 1, 10]}

# Initialize a GridSearchCV object that will perform the hyperparameter tuning
grid_search = GridSearchCV(model, parameters, cv=5)

# Fit the data to perform hyperparameter tuning
grid_search.fit(X_train, y_train)

# Print the best parameters
print("Best Parameters: ", grid_search.best_params_)

# Use the best model to make predictions
best_model = grid_search.best_estimator_
y_val_pred_svm2 = best_model.predict(X_val)

# Print classification report
print(classification_report(y_val, y_val_pred_svm2))

NameError: ignored

## 4. Model Testing

### Imported unlabeled test file

In [ ]:
df_unlabeled = pd.read_csv('unlabeled_test_test.txt', header=None, names=['Token'], sep='\t', error_bad_lines=False, quoting=3, skip_blank_lines=False)

# Display the first few rows
print(df_unlabeled.head())


# Replace NaN values with an empty string
df_unlabeled['Token'].fillna("", inplace=True)


num_rows = df_unlabeled.shape[0]
print("Number of rows: ", num_rows)

output_file_path = 'output.txt'

# Write the DataFrame to a CSV file
df_unlabeled.to_csv(output_file_path, index=False, header=False)


In [ ]:
# Transform the unlabeled data using the same vectorizer
X_unlabeled = tfidf_vectorizer.transform(df_unlabeled['Token'])

# Predict the POS tags of the unlabeled data
y_unlabeled_pred = nb.predict(X_unlabeled)

# Convert the predicted labels back to their original form
y_unlabeled_pred_tags = label_encoder.inverse_transform(y_unlabeled_pred)

# Create a DataFrame with the tokens and their predicted POS tags
df_test = pd.DataFrame({'Token': df_unlabeled['Token'], 'Predicted POS tag': y_unlabeled_pred_tags})

# Save the DataFrame to a text file
df_test.to_csv('teamname.test.txt', sep='\t', index=False)

## 5. Results Interpretation